In [9]:
import novibet_functions as nv
import stoiximan_function as stm
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

## Scrapping

In [4]:
# Set options for headless mode
options = webdriver.ChromeOptions()
options.add_argument("--headless")
# Set options for window size
options.add_argument("--window-size=1920,1200")

# Initialize the webdriver
driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)

C:\Users\panag\AppData\Local\Temp/ipykernel_17012/2731851925.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)


### 1. Novibet data

In [6]:
page_url = 'https://www.novibet.gr/en/sports'

# Football
football_string = nv.novibet_football_text(page_url, driver)
nv.novibet_football_export(football_string)

# Basketball
basketball_string = nv.novibet_basketball_text(driver)
nv.novibet_basketball_export(basketball_string)

# Tennis
tennis_string = nv.novibet_tennis_text(driver)
nv.novibet_tennis_export(tennis_string)

OSError: [Errno 22] Invalid argument: 'data\novibet_football.csv'

### 2. Stoiximan data

In [5]:
# Football
football_url = 'https://en.stoiximan.gr/sport/soccer/'
football_string = stm.stoiximan_football_text(football_url, driver)
stm.stoiximan_football_export(football_string)

# Basketball
basketball_url = 'https://en.stoiximan.gr/sport/basketball/'
basketball_string = stm.stoiximan_basketball_text(basketball_url, driver)
stm.stoiximan_basketball_export(basketball_string)

# Tennis
tennis_url = 'https://en.stoiximan.gr/sport/tennis/'
tennis_string = stm.stoiximan_tennis_text(tennis_url, driver)
stm.stoiximan_tennis_export(tennis_string)

## Calculations

In [4]:
football_novibet = pd.read_csv('data/novibet_football.csv')
football_stoiximan = pd.read_csv('data/stoiximan_football.csv')

In [5]:
football_novibet = football_novibet[['team1', 'team2','odds_1','odds_X','odds_2']]
football_stoiximan = football_stoiximan[['team1', 'team2','1','X','2']]

In [7]:
# Merge the two DataFrames on column 'A' to get the matching rows
merged_df = pd.merge(football_novibet, football_stoiximan, on='team1', how='inner')
len(merged_df)

265

In [14]:
merged_df

,team1,team2_x,odds_1,odds_X,odds_2,team2_y,1,X,2,max_1,max_X,max_2,arb
0,Scotland,Cyprus,1.24,6.00,13.25,Cyprus,1.21,5.70,15.50,1.24,6.00,15.50,1.037634
1,Armenia,Turkey,7.20,4.35,1.47,Turkey,7.80,4.10,1.44,7.80,4.35,1.47,1.038362
2,Belarus,Switzerland,17.00,6.50,1.19,Switzerland,18.00,6.00,1.18,18.00,6.50,1.19,1.049738
3,Israel,Kosovo,1.95,3.45,4.05,Kosovo,2.12,3.30,3.60,2.12,3.45,4.05,1.008467
4,Croatia U19,Sweden U19,1.70,3.85,4.15,Sweden U19,1.82,3.65,3.80,1.82,3.85,4.15,1.050155
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,AS Djerba,Kalaa Sport,2.95,2.65,2.55,Kalaa Sport,2.85,2.60,2.77,2.95,2.65,2.77,1.077352
261,Sporting Ben Arous,EGS Gafsa,4.05,3.00,1.89,Gawafel Gafsah,4.05,2.87,1.98,4.05,3.00,1.98,1.085297
262,Stade Gabesien,Croissant Sportif De Msaken,1.74,2.95,5.05,CS Msaken,1.82,2.90,4.85,1.82,2.95,5.05,1.086453
263,Bermuda,Guyana,2.20,3.35,3.10,Guyana,2.12,3.25,3.25,2.20,3.35,3.25,1.060745


In [11]:
merged_df['max_1'] = merged_df.apply(lambda row: max(row['odds_1'], row['1']), axis=1)
merged_df['max_X'] = merged_df.apply(lambda row: max(row['odds_X'], row['X']), axis=1)
merged_df['max_2'] = merged_df.apply(lambda row: max(row['odds_2'], row['2']), axis=1)

In [13]:
merged_df['arb'] = 1/merged_df['max_1'] + 1/merged_df['max_X'] + 1/merged_df['max_2']

In [15]:
mask = (merged_df['arb'] >= 0.85) & (merged_df['arb'] <= 1)
result_df = merged_df[mask]
result_df

,team1,team2_x,odds_1,odds_X,odds_2,team2_y,1,X,2,max_1,max_X,max_2,arb
60,Blackburn Rovers U21,Arsenal U21,2.60,3.75,2.20,Arsenal U21,2.02,3.70,3.10,2.60,3.75,3.10,0.973863
158,LKS Lodz II,Legionovia Legionowo,2.20,3.55,2.65,Legionovia Legionowo,1.65,3.80,4.25,2.20,3.80,4.25,0.952997
219,Loko Vltavin,Admira Praha,2.20,3.45,2.80,Admira Praha,3.50,3.35,1.95,3.50,3.45,2.80,0.932712
240,Uniao Suzano AC SP,EC Sao Jose,2.15,2.85,3.35,Sao Jose EC,2.87,3.05,2.42,2.87,3.05,3.35,0.974808
264,Vasco da Gama RJ (W),Fluminense RJ (W),2.55,3.55,2.30,Fluminense RJ (W),4.35,3.85,1.65,4.35,3.85,2.30,0.924408


In [39]:
merged_df['arb']

0      1.052391
1      1.004480
2      0.996692
3      0.091057
4      1.053492
         ...   
286    0.213033
287    0.074390
288    1.058029
289    0.305128
290    0.779221
Name: arb, Length: 291, dtype: float64

In [31]:
merged_df['1_y'].dtype

dtype('O')

In [34]:
football_stoiximan['1'] = football_stoiximan['1'].apply(pd.to_numeric, errors='coerce')
football_stoiximan['X'] = football_stoiximan['1'].apply(pd.to_numeric, errors='coerce')
football_stoiximan['2'] = football_stoiximan['1'].apply(pd.to_numeric, errors='coerce')

In [12]:
football_stoiximan

,date,time,team1,team2,1,X,2,O,O_odds,U,U_odds,Yes,Yes_odds,No,No_odds
0,24/03,21:45,Tiverton Town,Plymouth Parkway,2.52,3.55,2.45,O,1.62,U,2.10,Yes,1.55,No,2.27
1,24/03,21:45,Gibraltar,Greece,30.00,10.25,1.07,O,1.33,U,3.15,Yes,3.05,No,1.33
2,24/03,21:45,France,Netherlands,1.70,4.23,5.67,O,1.85,U,1.95,Yes,1.83,No,1.90
3,24/03,21:45,Sweden,Belgium,3.87,3.38,2.24,O,2.05,U,1.78,Yes,1.78,No,1.98
4,24/03,21:45,Moldova,Faroe Islands,2.37,2.95,3.20,O,2.57,U,1.47,Yes,2.10,No,1.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,25/03,18:00,Praiense,Rabo Peixe,2.50,2.95,2.80,O,2.25,U,1.57,No_bet,No_bet,No_bet,No_bet
889,25/03,18:00,DLR Waves (W),Athlone Town (W),3.05,3.30,2.15,O,1.87,U,1.83,No_bet,No_bet,No_bet,No_bet
890,25/03,18:00,Baltyk Gdynia,MKS Pogon Szczecin II,21.00,7.10,1.10,No_bet,No_bet,No_bet,No_bet,No_bet,No_bet,No_bet,No_bet
891,25/03,18:00,Montrose,Dunfermline Athletic,3.50,3.50,2.02,O,1.87,U,1.87,Yes,1.72,No,2.00


In [9]:
football_novibet

,championship,team1,team2,time,1,odds_1,X,odds_X,2,odds_2,Over,odds_over,Under,odds_under,GG,odds_gg,NG,odds_ng
0,Europe - EURO 2024 - Qualifying,Austria,Azerbaijan,21:45,1,1.22,X,6.20,2,14.00,O 2.5,1.70,U 2.5,2.15,GG,2.50,NG,1.52
1,Europe - EURO 2024 - Qualifying,Czech Republic,Poland,21:45,1,2.30,X,3.30,2,3.75,O 2.5,2.20,U 2.5,1.69,GG,1.89,NG,1.88
2,Europe - EURO 2024 - Qualifying,France,Netherlands,21:45,1,1.68,X,4.35,2,5.60,O 2.5,1.81,U 2.5,2.01,GG,1.84,NG,1.94
3,Europe - EURO 2024 - Qualifying,Gibraltar,Greece,21:45,1,41.00,X,10.75,2,1.07,O 3.5,2.10,U 3.5,1.74,GG,3.80,NG,1.26
4,Europe - EURO 2024 - Qualifying,Moldova,Faroe Islands,21:45,1,2.45,X,3.05,2,3.15,O 1.5,1.53,U 1.5,2.50,GG,2.15,NG,1.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,Central America - CONCACAF Nations League,Grenada,USA,Sat 02:00,1,41.00,X,11.00,2,1.05,O 3.5,1.64,U 3.5,2.20,GG,4.20,NG,1.20
648,Central America - CONCACAF Nations League,Nicaragua,Saint Vincent and the Grenadines,Sat 02:00,1,1.11,X,7.10,2,21.00,O 3.5,2.08,U 3.5,1.70,GG,2.60,NG,1.46
649,Central America - CONCACAF Nations League,Belize,Guatemala,Sat 04:00,1,10.00,X,5.10,2,1.29,O 2.5,1.75,U 2.5,2.05,GG,2.25,NG,1.60
650,Central America - CONCACAF Nations League,Bermuda,Guyana,Sat 20:00,1,2.20,X,3.35,2,3.10,O 2.5,1.97,U 2.5,1.81,GG,1.77,NG,1.99


In [ ]:
bet1 = 2.4
bet2 = 1.59

Arb_percentage = 1/bet1 + 1/bet2
print("Arb_per:  {0:.3f}".format(Arb_percentage))

win_amount = 50

total_outlay = (win_amount/bet1) + (win_amount/bet2)

profit = win_amount - total_outlay
roi = profit/total_outlay


print("Total_outlay:  {0:.2f}".format(total_outlay))
print("Bet on 1st: '  {0:.2f}".format(win_amount/bet1))
print("Bet on 2nd: '  {0:.2f}".format(win_amount/bet2))
print("Profit:        {0:.2f}".format(profit))
print("ROI:           {0:.2f}".format(roi))

## Testing 

In [7]:
football_string

"Daily coupon\nFootball\nBasketball\nTennis\n24 hours\n12 hours\n3 hours\nPopular First\nEurope - EURO 2024 - Qualifying\nAndorra\nRomania\nin 28'\n1\n16.75\nX\n6.00\n2\n1.21\nO 2.5\n1.91\nU 2.5\n1.90\nGG\n3.05\nNG\n1.36\n+470\nCroatia\nWales\nin 28'\n1\n1.40\nX\n4.40\n2\n9.10\nO 2.5\n2.08\nU 2.5\n1.76\nGG\n2.45\nNG\n1.53\n+553\nSpain\nNorway\nSO\nin 28'\n1\n1.43\nX\n5.15\n2\n9.00\nO 2.5\n1.84\nU 2.5\n1.98\nGG\n2.15\nNG\n1.69\n+601\nKazakhstan\nDenmark\nSun 16:00\n1\n13.25\nX\n5.50\n2\n1.26\nO 2.5\n1.64\nU 2.5\n2.25\nGG\n2.25\nNG\n1.64\n+482\nEngland\nUkraine\nSun 19:00\n1\n1.27\nX\n5.60\n2\n11.75\nO 2.5\n1.71\nU 2.5\n2.15\nGG\n2.30\nNG\n1.61\n+692\nLiechtenstein\nIceland\nSun 19:00\n1\n19.25\nX\n7.20\n2\n1.16\nO 2.5\n1.62\nU 2.5\n2.25\nGG\n2.75\nNG\n1.42\n+361\nSlovenia\nSan Marino\nSun 19:00\n1\n1.01\nX\n18.00\n2\n61.00\nO 4.5\n1.84\nU 4.5\n1.94\nGG\n3.30\nNG\n1.31\n+341\nLuxembourg\nPortugal\nSun 21:45\n1\n17.50\nX\n7.70\n2\n1.16\nO 3.5\n2.30\nU 3.5\n1.63\nGG\n2.50\nNG\n1.52\n+483\n

In [16]:
import novibet_functions as nvv

nvv.novibet_football_export(football_string)
football_string

OSError: [Errno 22] Invalid argument: 'data\novibet_football.csv'